# 🏁 Live Class Leaderboard (Streamlit + Google Sheets)  One-Notebook Setup

This notebook will create all files for a live classroom leaderboard with **Google Sheets persistence**.  
Run each cell in order. Cells marked **(RUN ME)** will write files to a project folder called `live_leaderboard/`.


In [1]:
# Creates the project folder structure
from pathlib import Path

BASE = Path.cwd() / "live_leaderboard"
(BASE / ".streamlit").mkdir(parents=True, exist_ok=True)

print("Project folder:", BASE)
print("Secrets folder:", BASE / ".streamlit")


Project folder: /home/michael-bruyns-haylett/GoogleDrive/PythonCode/Leaderboard_website/live_leaderboard
Secrets folder: /home/michael-bruyns-haylett/GoogleDrive/PythonCode/Leaderboard_website/live_leaderboard/.streamlit


In [2]:
from pathlib import Path

requirements = """streamlit==1.36.0
pandas>=2.1
altair>=5.0
gspread>=6.0.0
google-auth>=2.29.0
"""

Path("live_leaderboard/requirements.txt").write_text(requirements, encoding="utf-8")
print("Wrote live_leaderboard/requirements.txt")


Wrote live_leaderboard/requirements.txt


In [3]:
from pathlib import Path

app_code = r'''
import time
import pandas as pd
import altair as alt
import streamlit as st
from datetime import datetime

st.set_page_config(page_title="Class Leaderboard", page_icon="🏁", layout="wide")

# ----------------- Config -----------------
ADMIN_PASS = st.secrets.get("ADMIN_PASS", "teach")
GSHEETS_ENABLED = st.secrets.get("GSHEETS_ENABLED", False)
SHEET_URL = st.secrets.get("SHEET_URL", "")

# ----------------- Storage -----------------
class Storage:
    def __init__(self):
        self.mode = "csv"
        if GSHEETS_ENABLED and SHEET_URL:
            try:
                import gspread
                from google.oauth2.service_account import Credentials
                scope = [
                    "https://www.googleapis.com/auth/spreadsheets",
                    "https://www.googleapis.com/auth/drive",
                ]
                creds_info = st.secrets["gcp_service_account"]
                creds = Credentials.from_service_account_info(creds_info, scopes=scope)
                self.gc = gspread.authorize(creds)
                self.sh = self.gc.open_by_url(SHEET_URL)
                self.ws = self.sh.sheet1
                self.mode = "gsheets"
            except Exception as e:
                st.toast(f"⚠️ Google Sheets disabled ({e}). Using CSV.")
                self.mode = "csv"

        self.csv_path = "teams.csv"
        if self.mode == "csv":
            self.ensure_csv()

    def ensure_csv(self):
        import os
        if not os.path.exists(self.csv_path):
            seed = pd.DataFrame({
                "team": ["Alpha", "Bravo", "Charlie", "Delta"],
                "points": [0, 0, 0, 0],
                "badges": ["", "", "", ""],
                "streak": [0, 0, 0, 0],
                "last_update": ["", "", "", ""],
            })
            seed.to_csv(self.csv_path, index=False)

    def load(self) -> pd.DataFrame:
        if self.mode == "gsheets":
            rows = self.ws.get_all_records()
            df = pd.DataFrame(rows)
            if df.empty:
                df = pd.DataFrame(columns=["team","points","badges","streak","last_update"])
            if "points" in df.columns:
                df["points"] = pd.to_numeric(df["points"], errors="coerce").fillna(0).astype(int)
            if "streak" in df.columns:
                df["streak"] = pd.to_numeric(df["streak"], errors="coerce").fillna(0).astype(int)
            for c in ["team","badges","last_update"]:
                if c not in df.columns:
                    df[c] = ""
            return df
        else:
            return pd.read_csv(self.csv_path)

    def save(self, df: pd.DataFrame):
        df = df[["team","points","badges","streak","last_update"]].copy()
        if self.mode == "gsheets":
            values = [df.columns.tolist()] + df.astype(str).values.tolist()
            self.ws.clear()
            self.ws.update(values)
        else:
            df.to_csv(self.csv_path, index=False)

storage = Storage()

# ----------------- Data -----------------
@st.cache_data(ttl=2)
def load_data_cached():
    return storage.load()

df = load_data_cached().copy()

# ----------------- Auto-refresh -----------------
st.sidebar.toggle("Auto-refresh (every 3s)", value=True, key="ar")
if st.session_state.get("ar"):
    st.experimental_set_query_params(t=str(int(time.time()//3)))

# ----------------- Header -----------------
left, mid, right = st.columns([1.2, 1, 1])
with left:
    st.title("🏁 Live Class Leaderboard")
with mid:
    st.metric("Total Points", int(df["points"].sum()) if not df.empty else 0)
with right:
    top = df.sort_values("points", ascending=False).head(1)
    leader = "" if top.empty else f"{top.iloc[0]['team']} ({int(top.iloc[0]['points'])})"
    st.metric("🥇 Current Leader", leader)

# ----------------- Display -----------------
st.subheader("👀 Display Mode")
view = st.segmented_control("Style", ["Leaderboard Bars", "Podium", "Table"], default="Leaderboard Bars")

if view == "Leaderboard Bars":
    chart_df = df.sort_values("points", ascending=True)
    chart = (
        alt.Chart(chart_df)
        .mark_bar()
        .encode(x="points:Q", y="team:N", tooltip=["team","points","badges","streak","last_update"])
        .properties(height=400)
    )
    text = alt.Chart(chart_df).mark_text(align="left", dx=3).encode(
        x="points:Q", y="team:N", text="points:Q"
    )
    st.altair_chart(chart + text, use_container_width=True)
elif view == "Podium":
    podium = df.sort_values("points", ascending=False).head(3).reset_index(drop=True)
    cols = st.columns(3)
    places = ["🥇 1st","🥈 2nd","🥉 3rd"]
    for i in range(3):
        with cols[i]:
            if i < len(podium):
                row = podium.iloc[i]
                st.header(places[i]); st.subheader(row["team"]); st.metric("Points", int(row["points"]))
            else:
                st.header(places[i]); st.subheader(""); st.metric("Points",0)
else:
    st.dataframe(df.sort_values("points", ascending=False).reset_index(drop=True), hide_index=True, use_container_width=True)

# ----------------- Admin Panel -----------------
with st.expander("🔐 Admin Panel"):
    pwd = st.text_input("Enter admin passcode", type="password")
    if pwd == ADMIN_PASS:
        st.success("Admin mode active")
        new_team = st.text_input("Add team name")
        if st.button("➕ Add Team") and new_team.strip():
            df.loc[len(df)] = {"team":new_team,"points":0,"badges":"","streak":0,"last_update":datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
            storage.save(df); st.rerun()

        for idx,row in df.iterrows():
            c1,c2,c3,c4,c5,c6 = st.columns([2,1,1,1,1,2])
            with c1: st.write(f"**{row['team']}**  {row['points']} pts")
            with c2:
                if st.button("+1", key=f"p1{idx}"):
                    df.at[idx,"points"]=row["points"]+1; storage.save(df); st.balloons(); st.rerun()
            with c3:
                if st.button("+5", key=f"p5{idx}"):
                    df.at[idx,"points"]=row["points"]+5; storage.save(df); st.balloons(); st.rerun()
            with c4:
                if st.button("−1", key=f"m1{idx}"):
                    df.at[idx,"points"]=max(row["points"]-1,0); storage.save(df); st.rerun()
            with c6:
                badge=st.text_input("Add badge", key=f"bdg{idx}")
                if st.button("🏅 Add Badge", key=f"btn{idx}"):
                    df.at[idx,"badges"]=str(row["badges"])+" | "+badge; storage.save(df); st.rerun()
    else:
        st.info("Enter admin passcode to unlock admin panel")
'''

Path("live_leaderboard/app.py").write_text(app_code, encoding="utf-8")
print("Wrote live_leaderboard/app.py")


Wrote live_leaderboard/app.py


In [4]:
from pathlib import Path

secrets_template = """# Copy this to `.streamlit/secrets.toml` and fill in values.
# Do NOT commit the real secrets file to GitHub.

ADMIN_PASS = "teach"
GSHEETS_ENABLED = true
SHEET_URL = "https://docs.google.com/spreadsheets/d/YOUR_SHEET_ID/edit"

[gcp_service_account]
# Paste the FULL JSON from your Google Cloud service account key here.
# Keep quotes and newlines exactly as in the JSON.
"""

Path("live_leaderboard/.streamlit/secrets.TEMPLATE.toml").write_text(secrets_template, encoding="utf-8")
print("Wrote live_leaderboard/.streamlit/secrets.TEMPLATE.toml")


Wrote live_leaderboard/.streamlit/secrets.TEMPLATE.toml


## Google Cloud & Google Sheets Setup

1. **Create a Service Account**: Google Cloud Console → IAM & Admin → *Service Accounts* → **Create**.  
2. **Role**: *Editor* (or minimally enable **Google Sheets API** + **Google Drive API** and grant appropriate roles).  
3. **Keys**: Go to the service account → *Keys* → **Add key** → **Create new key** → **JSON** → download.  
4. **Google Sheet**: Create a blank sheet in Drive, copy the URL.  
5. **Share the Sheet** with the service account’s `client_email` (ends with `@...iam.gserviceaccount.com`) as **Editor**.  
6. **Create the real secrets file**:
   - Duplicate `live_leaderboard/.streamlit/secrets.TEMPLATE.toml` to `live_leaderboard/.streamlit/secrets.toml`.
   - Paste your JSON into the `[gcp_service_account]` section.
   - Replace `SHEET_URL` with your sheet’s URL.


In [5]:
import os

for root, dirs, files in os.walk("live_leaderboard"):
    level = root.replace("live_leaderboard", "").count(os.sep)
    indent = "  " * level
    print(f"{indent}{os.path.basename(root)}/")
    for f in files:
        print(f"{indent}  {f}")


live_leaderboard/
  app.py
  requirements.txt
  .streamlit/
    secrets.TEMPLATE.toml


## Run Locally (Terminal commands to copy/paste)

```bash
cd live_leaderboard
python -m venv .venv
# Windows:
.venv\Scripts\activate
# macOS/Linux:
source .venv/bin/activate

pip install -r requirements.txt
streamlit run app.py


## Cell 9  Deploy to Streamlit Community Cloud (Markdown)
```markdown
## Optional: Deploy to Streamlit Community Cloud (free)

1. Push the `live_leaderboard/` folder to GitHub.  
   - **Do NOT** commit `.streamlit/secrets.toml`.  
2. Go to https://streamlit.io/cloud → **New app** → select your repo and `app.py`.  
3. In **App → Settings → Secrets**, paste the **same TOML** from your local `secrets.toml`.  
4. Deploy → share the public URL (e.g., `https://your-app-name.streamlit.app`) with students.
